In [1]:
import controllables.energyplus as _ooep_
from energyplus.dataset.basic import dataset as _epds_

simulator = _ooep_.World(
    input=_ooep_.World.Specs.Input(
        world='tmp_timestep 10 min.idf',
        weather='SGP_Singapore_486980_IWEC.epw',
    ),
    # output=_ooep_.World.Specs.Output(
    #     report='./tmp',
    # ),
    runtime=_ooep_.World.Specs.Runtime(
        recurring=True,
        #design_day=True,
    ),
)

# add progress provider
_ = simulator.add('logging:progress')

  0%|          | 0/100 [00:00<?, ?it/s]

In [2]:
_ = simulator.awaitable.run()

In [3]:
simulator.variables.available_keys()

In [1]:
import numpy as _numpy_
import gymnasium as _gymnasium_
import pandas as pd
import os

from controllables.core.tools.gymnasium import (
    BoxSpace,
    DictSpace,
)
from controllables.core.tools.ray import (
    ExternalEnv,
)

import ray
from ray import tune , air
from ray.tune.schedulers import PopulationBasedTraining

from controllables.energyplus import (
    World,
    Actuator,
    OutputVariable,
)
import pythermalcomfort as pytc
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.impala import ImpalaConfig
from ray.rllib.algorithms.sac import SACConfig

from ray.rllib.algorithms.callbacks import DefaultCallbacks

import logging
import csv
import math

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', filename='training.log')

class CustomExternalEnv(ExternalEnv):
    def __init__(self, env_config):
        pass

    def step(self, action):
        pass

class RewardFunction:
    def __init__(self, metab_rate=1.5, clothing=.5, pmv_limit=.5):
        self._metab_rate = _numpy_.asarray(metab_rate)
        self._clothing = _numpy_.asarray(clothing)
        self._pmv_limit = _numpy_.asarray(pmv_limit)
    
   
    def __call__(self, agent):
        observation = agent.observation.value     
        AHU_COOLING_COIL = observation['AHU COOLING COIL']
        Fan_Electricity_Rate = observation['Fan Electricity Rate']
        Office_Occupancy = observation['Office Occupancy']
        pmv = pytc.models.pmv_ppd(
            tdb=(tdb := observation['temperature:drybulb']), 
            tr=observation['temperature:radiant'], 
            # calculate relative air speed
            vr=pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate), 
            rh=observation['humidity'], 
            met=self._metab_rate, 
            # calculate dynamic clothing
            clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
            limit_inputs=False ,
        )['pmv']
        
        reward = (
            2*Office_Occupancy*((self._pmv_limit - _numpy_.abs(pmv)) / self._pmv_limit) - (AHU_COOLING_COIL/180000) 
            - (Fan_Electricity_Rate/6200)
        )
        inputs = {'AHU_energy': AHU_COOLING_COIL, 'tdb': observation['temperature:drybulb'],
                'tr': observation['temperature:radiant'], 
                'vr': pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate), 
                'rh': observation['humidity'], 'met': self._metab_rate,
                'clo': pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
                'pmv':pmv ,
                'reward':reward}
        for name, value in inputs.items():
            if math.isnan(value):
                print(f"NaN detected in input: {name}")
                print(dict(
                    tdb=(observation['temperature:drybulb']), 
                    tr=observation['temperature:radiant'], 
                    # calculate relative air speed
                    vr=pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate), 
                    rh=observation['humidity'], 
                    met=self._metab_rate, 
                    # calculate dynamic clothing
                    clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
                ))
                raise Exception('TODO')
        if math.isnan(reward):
            reward = 0


        return reward



from ray.rllib.algorithms.callbacks import DefaultCallbacks


class TraceCallbacks(DefaultCallbacks):
    def on_episode_start(self, *, worker, episode, base_env, **kwargs) -> None:
        from energyplus.ooep.specs.tools import VariableHistory

        env = worker.env

        episode._user_history = VariableHistory()
        display(
            episode._user_history.plot({
                'traces': [{
                    'x': env.world['wallclock:calendar'], 
                    'y': env.reward,
                }],
            }, autoupdate=1_000)
        )

    def on_episode_step(self, *, worker, episode, base_env, **kwargs) -> None:
        episode._user_history.poll()


config = (
    PPOConfig()
    .environment(
        ExternalEnv,
        env_config=ExternalEnv.Config(
            action_space=DictSpace({
                'thermostat_1FWEST': BoxSpace(
                    low=22., high=30.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Schedule:Compact',
                    control_type='Schedule Value',
                    key='1FFIRSTFLOORWEST:OPENOFFICE COOLING SETPOINT SCHEDULE',
                )),
                'thermostat_1FEAST': BoxSpace(
                    low=22., high=30.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Schedule:Compact',
                    control_type='Schedule Value',
                    key='1FFIRSTFLOOREAST:OPENOFFICE',
                )),

                # 'thermostat_1': BoxSpace(
                #     low=22., high=30.,
                #     dtype=_numpy_.float32,
                #     shape=(),
                # ).bind(Actuator.Ref(
                #     type='Schedule:Compact',
                #     control_type='Schedule Value',
                #     key='1FFIRSTFLOOREAST:OPENOFFICE COOLING SETPOINT SCHEDULE',
                # )),
            }),    
            observation_space=DictSpace({
                'temperature:drybulb': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Air Temperature',
                    key='1FFIRSTFLOORWEST:OPENOFFICE',
                )),
                'temperature:radiant': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Radiant Temperature',
                    key='1FFIRSTFLOORWEST:OPENOFFICE',
                )),
                'humidity': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Air Relative Humidity',
                    key='1FFIRSTFLOORWEST:OPENOFFICE',
                )),
                'AHU COOLING COIL': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Cooling Coil Total Cooling Rate',
                    key='AIR LOOP AHU COOLING COIL',
                )),
                'Fan Electricity Rate': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Fan Electricity Rate',
                    key='AIR LOOP AHU SUPPLY FAN',
                )),
                'Office Occupancy':BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Schedule Value',
                    key='Office_OpenOff_Occ',
                )),
            }),
            reward_function=RewardFunction(),
            episode_events={
                'step': 'begin_zone_timestep_after_init_heat_balance',
            },
            system=lambda: World(
                input=World.Specs.Input(
                    world='tmp_timestep 10 min.idf',
                    weather='SGP_Singapore_486980_IWEC.epw',
                ),
                output=World.Specs.Output(
                    #report='./tmp',
                ),
                runtime=World.Specs.Runtime(
                    recurring=True,
                #     #design_day=True,
                ),
            ),
            system_mgmt_enabled=True,
        ),  

    )
    .rollouts(
        enable_connectors=False,

    )
    .framework("torch")

    
)
pbt = PopulationBasedTraining(
    time_attr="training_iteration",
    perturbation_interval=5,  # Number of iterations between hyperparameter adjustments
    resample_probability=0.25,  # Probability of resampling a hyperparameter
    hyperparam_mutations={
        "lr": tune.uniform(1e-5, 0.1),
        "train_batch_size": [10_000],
        "sgd_minibatch_size": [32, 64, 128, 256, 512],
        "num_sgd_iter": [10, 20, 30],
        "clip_param": tune.uniform(0.1, 0.3),
        # Add more hyperparameters if needed
    }
)

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    tune_config=tune.TuneConfig(
        scheduler=pbt,
        num_samples=4,  # 群体大小
        metric="episode_reward_mean",  # 同样的指标
        mode="max",  # 同样的优化方向
    ),
    run_config=air.RunConfig(
        stop={"training_iteration": 200},  # 训练停止条件
        checkpoint_config=air.CheckpointConfig(
        checkpoint_at_end=True  # 在训练结束时保存检查点
        )
    )
)

results = tuner.fit()
best_result = results.get_best_result()
print("Best Hyperparameters found: ", best_result)

/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1159: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-09-30_09-38-14_995899_2615718/logs/gcs_server.out' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1159: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-09-30_09-38-14_995899_2615718/logs/gcs_server.err' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1164: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-09-30_09-38-14_995899_2615718/logs/monitor.out' mode='a' encoding='utf-8'>
  self.start_monitor()
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1164: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session

(PPO pid=2617240) 2024-09-30 09:38:23,251	WARNING algorithm_config.py:643 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=2617240) 2024-09-30 09:38:23,253	INFO algorithm.py:536 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=2617523) 2024-09-30 09:38:29,333	WARNING env.py:85 -- Env checking isn't implemented for RemoteBaseEnvs, ExternalMultiAgentEnv, ExternalEnvs or environments that are Ray actors.
(RolloutWorker pid=2617523) /home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/rllib/models/preprocessors.py:307: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
(RolloutWorker pid=2617523)   preprocessor = preprocessor_class(space, self._options)
(RolloutWorker pid=2617523) /home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/rllib/m

In [ ]:
import numpy as _numpy_
import gymnasium as _gymnasium_
import pandas as pd
import os

from controllables.core.tools.gymnasium import (
    BoxSpace,
    DictSpace,
)
from controllables.core.tools.ray import (
    ExternalEnv,
)

import ray
from ray import tune , air
from ray.tune.schedulers import PopulationBasedTraining

from controllables.energyplus import (
    World,
    Actuator,
    OutputVariable,
)
import pythermalcomfort as pytc
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.impala import ImpalaConfig
from ray.rllib.algorithms.sac import SACConfig

from ray.rllib.algorithms.callbacks import DefaultCallbacks

import logging
import csv
import math

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', filename='training.log')

class CustomExternalEnv(ExternalEnv):
    def __init__(self, env_config):
        pass

    def step(self, action):
        pass

class RewardFunction:
    def __init__(self, metab_rate=1.5, clothing=.5, pmv_limit=.5):
        self._metab_rate = _numpy_.asarray(metab_rate)
        self._clothing = _numpy_.asarray(clothing)
        self._pmv_limit = _numpy_.asarray(pmv_limit)
    
   
    def __call__(self, agent):
        observation = agent.observation.value     
        AHU_COOLING_COIL = observation['AHU COOLING COIL']
        Fan_Electricity_Rate = observation['Fan Electricity Rate']
        Office_Occupancy = observation['Office Occupancy']
        pmv = pytc.models.pmv_ppd(
            tdb=(tdb := observation['temperature:drybulb']), 
            tr=observation['temperature:radiant'], 
            # calculate relative air speed
            vr=pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate), 
            rh=observation['humidity'], 
            met=self._metab_rate, 
            # calculate dynamic clothing
            clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
            limit_inputs=False ,
        )['pmv']
        
        reward = (
            2*Office_Occupancy*((self._pmv_limit - _numpy_.abs(pmv)) / self._pmv_limit) - (AHU_COOLING_COIL/180000) 
            - (Fan_Electricity_Rate/6200)
        )
        inputs = {'AHU_energy': AHU_COOLING_COIL, 'tdb': observation['temperature:drybulb'],
                'tr': observation['temperature:radiant'], 
                'vr': pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate), 
                'rh': observation['humidity'], 'met': self._metab_rate,
                'clo': pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
                'pmv':pmv ,
                'reward':reward}
        for name, value in inputs.items():
            if math.isnan(value):
                print(f"NaN detected in input: {name}")
                print(dict(
                    tdb=(observation['temperature:drybulb']), 
                    tr=observation['temperature:radiant'], 
                    # calculate relative air speed
                    vr=pytc.utilities.v_relative(v=observation.get('airspeed', .1), met=self._metab_rate), 
                    rh=observation['humidity'], 
                    met=self._metab_rate, 
                    # calculate dynamic clothing
                    clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
                ))
                raise Exception('TODO')
        if math.isnan(reward):
            reward = 0


        return reward



from ray.rllib.algorithms.callbacks import DefaultCallbacks


class TraceCallbacks(DefaultCallbacks):
    def on_episode_start(self, *, worker, episode, base_env, **kwargs) -> None:
        from energyplus.ooep.specs.tools import VariableHistory

        env = worker.env

        episode._user_history = VariableHistory()
        display(
            episode._user_history.plot({
                'traces': [{
                    'x': env.world['wallclock:calendar'], 
                    'y': env.reward,
                }],
            }, autoupdate=1_000)
        )

    def on_episode_step(self, *, worker, episode, base_env, **kwargs) -> None:
        episode._user_history.poll()


config = (
    PPOConfig()
    .environment(
        ExternalEnv,
        env_config=ExternalEnv.Config(
            action_space=DictSpace({
                'thermostat_1FWEST': BoxSpace(
                    low=22., high=30.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Schedule:Compact',
                    control_type='Schedule Value',
                    key='1FFIRSTFLOORWEST:OPENOFFICE COOLING SETPOINT SCHEDULE',
                )),
                'thermostat_1FEAST': BoxSpace(
                    low=22., high=30.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Schedule:Compact',
                    control_type='Schedule Value',
                    key='1FFIRSTFLOOREAST:OPENOFFICE',
                )),

                # 'thermostat_1': BoxSpace(
                #     low=22., high=30.,
                #     dtype=_numpy_.float32,
                #     shape=(),
                # ).bind(Actuator.Ref(  File "/home/user@AD/lab/reports/apr01/.venv/lib/python3.11/site-packages/controllables/energyplus/events.py", line 200, in _state
                #     type='Schedule:Compact',
                #     control_type='Schedule Value',
                #     key='1FFIRSTFLOOREAST:OPENOFFICE COOLING SETPOINT SCHEDULE',
                # )),
            }),    
            observation_space=DictSpace({
                'temperature:drybulb': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Air Temperature',
                    key='1FFIRSTFLOORWEST:OPENOFFICE',
                )),
                'temperature:radiant': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Mean Radiant Temperature',
                    key='1FFIRSTFLOORWEST:OPENOFFICE',
                )),
                'humidity': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Zone Air Relative Humidity',
                    key='1FFIRSTFLOORWEST:OPENOFFICE',
                )),
                'AHU COOLING COIL': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Cooling Coil Total Cooling Rate',
                    key='AIR LOOP AHU COOLING COIL',
                )),
                'Fan Electricity Rate': BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Fan Electricity Rate',
                    key='AIR LOOP AHU SUPPLY FAN',
                )),
                'Office Occupancy':BoxSpace(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='Schedule Value',
                    key='Office_OpenOff_Occ',
                )),
            }),
            reward_function=RewardFunction(),
            episode_events={
                'step': 'begin_zone_timestep_after_init_heat_balance',
            },
            system=simulator
        ),  

    )
    .rollouts(
        num_rollout_workers=0,
        enable_connectors=False,

    )
    .framework("torch")    
)

In [ ]:
from energyplus.ooep.addons.logging import VariableLogger

varlogger = (
    VariableLogger(
        dict(maxlen=10_000),
        autoupdate='begin_zone_timestep_after_init_heat_balance',
    )
    .__attach__(simulator)
)

# varlogger.plot({
#     'traces': [
#         dict(
#             x='wallclock:calendar', 
#             y=Actuator.Ref(
#                 type='Fan',
#                 control_type='Fan Air Mass Flow Rate',
#                 key='AIR LOOP AHU SUPPLY FAN',
#             ),
#         ),        
#     ],
# })


In [ ]:
varlogger.track('clock',
    'wallclock:calendar'
)

varlogger.track('Fan Air Mass Flow Rate', Actuator.Ref(
    type='Fan',
    control_type='Fan Air Mass Flow Rate',
    key='AIR LOOP AHU SUPPLY FAN',
),
)

varlogger.track('thermostat_1FEAST',Actuator.Ref(
    type='Schedule:Compact',
    control_type='Schedule Value',
    key='1FFIRSTFLOOREAST:OPENOFFICE',
)
)

varlogger.track('thermostat_1FWEST',Actuator.Ref(
    type='Schedule:Compact',
    control_type='Schedule Value',
    key='1FFIRSTFLOORWEST:OPENOFFICE COOLING SETPOINT SCHEDULE',
)
)

varlogger.track('Zone Mean Air Temperature',OutputVariable.Ref(
    type='Zone Mean Air Temperature',
    key='1FFIRSTFLOORWEST:OPENOFFICE',
)
)

varlogger.track(OutputVariable.Ref(
    type='Zone Air Relative Humidity',
    key='1FFIRSTFLOORWEST:OPENOFFICE',
)
)

varlogger.track('Zone Mean Radiant Temperature',OutputVariable.Ref(
    type='Zone Mean Radiant Temperature',
    key='1FFIRSTFLOORWEST:OPENOFFICE',
)
)

varlogger.track('Cooling Coil Total Cooling Rate',OutputVariable.Ref(
    type='Cooling Coil Total Cooling Rate',
    key='AIR LOOP AHU COOLING COIL',
)
)

varlogger.track('Fan Electricity Rate',OutputVariable.Ref(
    type='Fan Electricity Rate',
    key='AIR LOOP AHU SUPPLY FAN',
)
)

varlogger.track('Office Occupancy',OutputVariable.Ref(
    type='Schedule Value',
    key='Office_OpenOff_Occ',
)
)


In [ ]:
import pandas as pd
df = varlogger.dataframe()
df['clock'] = pd.to_datetime(df['clock'])
sorted_time = df['clock'].sort_values().reset_index(drop=True)
df['clock'] = sorted_time
import itables as _itables_
_itables_.show(df)
# df.to_csv('datasave/data.csv', index=False, sep=';')
